## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [0]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
# 讀取糖尿病資料集
diabetes = datasets.load_diabetes()

In [0]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(diabetes.data, diabetes.target, test_size = 0.2, random_state =42) 

In [0]:
# 建立模型
clf = GradientBoostingRegressor()

In [0]:
#訓練模型
clf.fit(x_train, y_train)

#預測模型
y_pred = clf.predict(x_test)

In [8]:
scr = metrics.mean_squared_error(y_test, y_pred)
print('Scr: ', scr)

Scr:  2920.8039474514635


In [9]:
#树的数量（n_estimators）
#每棵独立树的深度(max_depth)
#损失函数(loss)
clf

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150, 200, 250, 300, 500]
max_depth = [1, 3, 5, 7, 9, 9, 11]
para_grid = dict(n_estimators = n_estimators, max_depth = max_depth)

In [0]:
## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, para_grid, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

In [16]:
# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 49 candidates, totalling 147 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 101 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 147 out of 147 | elapsed:   15.2s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [17]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" %(grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -3324.465984 using {'max_depth': 1, 'n_estimators': 100}


In [0]:
# 使用最佳參數重新建立模型
clf_best =  GradientBoostingRegressor(max_depth = grid_result.best_params_['max_depth'],
                                     n_estimators = grid_result.best_params_['n_estimators'])

In [0]:
# 訓練模型
clf_best.fit(x_train, y_train)

# 預測模型
y_pred = clf_best.predict(x_test)

In [30]:
scr = metrics.mean_squared_error(y_test, y_pred)
print('Scr: ', scr)

Scr:  2751.5309446807805
